# Introduction

Most of the post-processing material presented here is written in python, and makes extensive use of the numpy package for rapid computation on large data arrays.
Here we show two important basics of python/numpy in the context of investigating your COMPAS simulation.

## Material

### [1. Inspecting the data ](#1.-Inspecting-the-data)
TODO:! Look at the raw hdf5 data to see which parameters are available and check that it matches expectations.

### [2. Slicing the data ](#2.-Slicing-the-data)
Select specific systems and their parameters using seeds.

### [3. Visualizing the data ](#3.-Visualizing-the-data)
Binning and visualising your data.

### For the following sections, you will need to have the following packages installed.
### `numpy, h5py, time, matplotlib`

In [13]:
#python libraries
import os, sys
import numpy as np               # for handling arrays
import h5py as h5                # for reading the COMPAS data
import time                      # for finding computation time
import matplotlib.pyplot as plt  #for plotting

# Import COMPAS specific scripts
sys.path.append(compasRootDir + 'postProcessing/PythonScripts')
from compasUtils import printCompasDetails

# This is known as an ipython magic command, and allows plots to be produced within the notebook
%matplotlib inline

# Choose an output hdf5 file to work with
compasRootDir = os.environ['COMPAS_ROOT_DIR'] 
pathToData = compasRootDir + 'postProcessing/Tutorial/COMPAS_Output/COMPAS_Output.h5'


# 1. Inspecting the data

Often the first thing you want to do with new data is simply to look at it! Getting familiar with the data, including available parameters, size of the data file, etc. will help to inform how best to proceed with the analysis. We provide several useful functions for inspecting the data, `printCompasDetails`, `getEventHistory`, and `getEventStrings`.

--

If you do not already have a COMPAS_Output.h5 ready, see Section 1 [Working With HDF5](./WorkingWithHDF5.py) on how to create your own output file, or download some data from our [Zenodo database](https://zenodo.org/communities/compas/?page=1&size=20).

*Note:* These cells may take a long time if you test them on large datasets.

In [5]:
Data  = h5.File(pathToData)
print(list(Data.keys()))

['BSE_Common_Envelopes', 'BSE_Double_Compact_Objects', 'BSE_RLOF', 'BSE_Supernovae', 'BSE_System_Parameters', 'Run_Details']


The output above represents the event categories available from the particular run. If you used the output produced in the previous tutorial, you should see `['BSE_Common_Envelopes', 'BSE_Double_Compact_Objects', 'BSE_RLOF', 'BSE_Supernovae', 'BSE_System_Parameters', 'Run_Details']`. Note that for smaller runs which do not produce any of a particular type of output, the output category will not be created. 

Brief description of the categories:
- 'BSE_System_Parameters': Initial state of the binary
- 'BSE_RLOF': Any mass transfer events that occured within the binary
- 'BSE_Common_Envelopes': If any of the mass transfer events were unstable, details will be included here.
- 'BSE_Supernovae': Parameters and outcome of any supernovae that occured in the binary
- 'BSE_Double_Compact_Objects': Includes key information of all binaries which end their lives as an intact pair of compact obects (either neutron stars or black holes)
- 'Run_Details': Information on the input settings supplied to the Compas run

To extract the data from these categories, we use the following syntax

In [8]:
SPs = Data['BSE_System_Parameters']
MTs = Data['BSE_RLOF']
CEs = Data['BSE_Common_Envelopes']
SNe = Data['BSE_Supernovae']
DCs = Data['BSE_Double_Compact_Objects']

Each of these is a dictionary mapping parameter names (keys) to an array of values

In [9]:
print(SPs.keys())

<KeysViewHDF5 ['CE_Alpha', 'CH_on_MS(1)', 'CH_on_MS(2)', 'Eccentricity@ZAMS', 'Equilibrated_At_Birth', 'Error', 'LBV_Factor', 'Mass@ZAMS(1)', 'Mass@ZAMS(2)', 'Merger', 'Merger_At_Birth', 'Metallicity@ZAMS(1)', 'Metallicity@ZAMS(2)', 'Omega@ZAMS(1)', 'Omega@ZAMS(2)', 'SEED', 'SN_Kick_Magnitude_Random_Number(1)', 'SN_Kick_Magnitude_Random_Number(2)', 'SemiMajorAxis@ZAMS', 'Sigma_Kick_CCSN_BH', 'Sigma_Kick_CCSN_NS', 'Sigma_Kick_ECSN', 'Sigma_Kick_USSN', 'Stellar_Type(1)', 'Stellar_Type(2)', 'Stellar_Type@ZAMS(1)', 'Stellar_Type@ZAMS(2)', 'Unbound', 'WR_Factor']>


If we want to view, say, the random seeds in the system parameters file, we run

In [11]:
spSeeds = SPs['SEED'][()]
print(spSeeds)

[1636090177 1636090178 1636090179 1636090180 1636090181 1636090182
 1636090183 1636090184 1636090185 1636090186 1636090187 1636090188
 1636090189 1636090190 1636090191 1636090192 1636090193 1636090194
 1636090195 1636090196 1636090197 1636090198 1636090199 1636090200
 1636090201 1636090202 1636090203 1636090204 1636090205 1636090206
 1636090207 1636090208 1636090209 1636090210 1636090211 1636090212
 1636090213 1636090214 1636090215 1636090216 1636090217 1636090218
 1636090219 1636090220 1636090221 1636090222 1636090223 1636090224
 1636090225 1636090226 1636090227 1636090228 1636090229 1636090230
 1636090231 1636090232 1636090233 1636090234 1636090235 1636090236
 1636090237 1636090238 1636090239 1636090240 1636090241 1636090242
 1636090243 1636090244 1636090245 1636090246 1636090247 1636090248
 1636090249 1636090250 1636090251 1636090252 1636090253 1636090254
 1636090255 1636090256 1636090257 1636090258 1636090259 1636090260
 1636090261 1636090262 1636090263 1636090264 1636090265 163609

This is useful for extracting the arrays of single parameters, but for a more convenient view of the whole system parameters file, we can use the `printCompasDetails` function.

In [12]:
printCompasDetails(SPs) # Note - the output of this is a pandas dataframe

SEED,(units),1636090177,1636090178,1636090179,1636090180,1636090181,1636090182,1636090183,1636090184,1636090185,...,1636091167,1636091168,1636091169,1636091170,1636091171,1636091172,1636091173,1636091174,1636091175,1636091176
CE_Alpha,-,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CH_on_MS(1),State,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CH_on_MS(2),State,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Eccentricity@ZAMS,-,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Equilibrated_At_Birth,Event,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Error,-,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LBV_Factor,-,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,...,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000
Mass@ZAMS(1),Msol,15.150945,7.303783,5.115267,6.148296,12.360567,9.409131,12.492657,16.399265,8.723310,...,8.803164,10.762356,5.513340,7.444313,8.203689,21.979733,11.785701,12.654168,7.288199,9.392270
Mass@ZAMS(2),Msol,4.360026,6.176230,1.061175,5.398415,3.095323,3.089870,4.059830,15.634044,1.082531,...,3.733252,5.639837,1.654975,0.350391,1.367229,0.610634,0.585199,1.536171,7.062301,0.300414
Merger,Event,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000


PrintCompasDetails optionally also takes seeds and/or a boolean mask as arguments, to help filter the data.

Some of the output categories have multiple events for a single seed (e.g `BSE_RLOF` if there are multiple mass transfer events). Using both seeds and mask inputs can help to extract a specifc event from several for that seed. 

In [28]:
# Example: out of the first 20 seeds, only show systems which end as a merger

mtSeeds = MTs['SEED'][()]
sds, cts = np.unique(mtSeeds, return_counts=True)
sds[cts>5]
printCompasDetails(MTs, 1636090318)

SEED,(units),1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,...,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318,1636090318
CEE>MT,State,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Eccentricity<MT,-,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.743900
Eccentricity>MT,-,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.743900,0.000000
MT_Event_Counter,Count,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,24.000000,25.000000,26.000000,27.000000,28.000000,29.000000,30.000000,31.000000,32.000000,33.000000
Mass(1)<MT,Msol,11.773512,3.942982,4.952011,4.957002,4.957069,4.957070,4.957071,4.957071,4.957071,...,4.957074,4.957074,4.957075,4.957075,4.957075,4.957075,4.957075,4.957075,4.957076,1.360462
Mass(1)>MT,Msol,3.942982,4.952011,4.957002,4.957069,4.957070,4.957071,4.957071,4.957071,4.957071,...,4.957074,4.957075,4.957075,4.957075,4.957075,4.957075,4.957075,4.957076,1.360462,1.360462
Mass(2)<MT,Msol,2.403020,2.403020,1.393991,1.389000,1.388933,1.388932,1.388931,1.388931,1.388931,...,1.388928,1.388927,1.388927,1.388927,1.388927,1.388927,1.388927,1.388926,1.388926,1.388926
Mass(2)>MT,Msol,2.403020,1.393991,1.389000,1.388933,1.388932,1.388931,1.388931,1.388931,1.388931,...,1.388927,1.388927,1.388927,1.388927,1.388927,1.388927,1.388926,1.388926,1.388926,1.388926
RLOF(1)<MT,Event,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RLOF(1)>MT,State,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# 2. Slicing the data

One of the most important numbers in the COMPAS output is the system seed. The seed represents the unique identifier to a specific system in a simulation. It is also used as the seed value in random number generation, which is useful when trying to reproduce a given system identically. Therefore the properties and events of a single binary system can be recovered by looking at its seed across different output categories. 

Here we introduce the basics of manipulating the data using the seeds. We provide an example on how we get the initial parameters of systems that ended up forming double compact objects.

Naively, we might try to use For Loops with Conditions to extract systems of interest to a list. However, this can potentially be computationally expensive.

Here we present a method to more efficiently 'slice' the data using numpy and boolean masks. These are slightly more involved but are computationally quick and use intuitive logic.

## Question: What were the initial total masses of the double compact objects?

In [ ]:
def calculateTotalMassesNaive(pathData=None):
    Data  = h5.File(pathToData)
    
    totalMasses = []
    
    #for syntax see section 1 
    seedsDCOs     = Data['DoubleCompactObjects']['SEED'][()]
    
    #get info from ZAMS
    seedsSystems  = Data['SystemParameters']['SEED'][()]
    M1ZAMSs       = Data['SystemParameters']['Mass@ZAMS_1'][()]
    M2ZAMSs       = Data['SystemParameters']['Mass@ZAMS_2'][()]

    for seedDCO in seedsDCOs:
        for nrseed in range(len(seedsSystems)):
            seedSystem = seedsSystems[nrseed]
            if seedSystem == seedDCO:
                M1 = M1ZAMSs[nrseed]
                M2 = M2ZAMSs[nrseed]
                Mtot = M1 + M2
                totalMasses.append(Mtot)

    Data.close()
    return totalMasses

In [ ]:
# calculate function run time
start   = time.time()
MtotOld = calculateTotalMassesNaive(pathData=pathToData)
end     = time.time()
timeDiffNaive = end-start

print('%s seconds, using for loops.' %(timeDiffNaive)) 

# Optimizing the above loop

## 0 - Use built-in numpy routines

Numpy arrays can make use of a powerful library of optimization tools which allow the user to bypass computationally heavy for loops. 

For example, we can speed up the calculation of the element-wise sum of two arrays with:

In [ ]:
M1ZAMS  = Data['SystemParameters']['Mass@ZAMS_1'][()]
M2ZAMS  = Data['SystemParameters']['Mass@ZAMS_2'][()]
    
mTotalAllSystems  = np.add(M1ZAMS, M2ZAMS)

## 1 - Use boolean masks in a single file

There is a useful trick for when you want only those elements which satisfy a specific condition. 

Where previously we put the condition in an if statement nested within a for loop, now we will use an array of booleans to mask out the undesired elements. 

The boolean array will have the same length as the input array, with 

In [ ]:
# Create a boolean array from the total mass array which is True
# if the total mass of the corrresponding system is less than 40. 

maskMtot = (mTotalAllSystems <= 40)

**Crucially, you can apply this mask to all other columns in the same file because, by construction, they all have the same length.**

In [ ]:
# seeds of systems with total mass below 40
seeds  = Data['SystemParameters']['SEED'][()]
seedsMtotBelow40 = seeds[maskMtot]

Note that this works because the order of the two columns (seeds and total masses) are the same. 

For example, the total mass of the third system entry corresponds to the seed at the third system entry.

## 2 - Use seeds as masks between files

### Example 1

Before we continue it is useful to understand how the COMPAS-popsynth printing works.

Each simulated system will be initialized only once and so will have only one line in the SystemParameters file. However, lines in CommonEnvelopes are created whenever a system goes through CE, which might happen multiple times for a single system, or potentially not at all. Similarly, in the Supernovae file, you will find at most two lines per system, but possibly none. DoubleCompactObject lines are printed only when both remnants are either Neutron Stars or Black Holes (but also includes disrupted systems), which happens at most once per system. 

For this reason, it is in general not the case that the system on line $n$ of one file corresponds will match the system on line $n$ of another file.

In order to match systems across files, we need to extract the seeds of desired systems from one file, and apply them as a mask in the other file. 

In [ ]:
# example mock data from two files
SystemSeeds = np.array([1,  2,  3,  4 ])
SystemMass1 = np.array([1, 20,  5, 45 ])
DCOSeeds    = np.array([    2,      4 ])

# Calculate mask for which elements of SystemSeeds are found in DCOSeeds - see numpy.in1d documentation for details
mask = np.in1d(SystemSeeds, DCOSeeds)

print(mask)
print(SystemSeeds[mask])
print(SystemMass1[mask])

# Optimized loop

In [ ]:
def calculateTotalMassesOptimized(pathData=None):
    Data  = h5.File(pathToData)
    
    totalMasses = []
    
    #for syntax see section 1 with basic syntax
    seedsDCOs     = Data['DoubleCompactObjects']['SEED'][()]
    #get info from ZAMS
    seedsSystems  = Data['SystemParameters']['SEED'][()]
    M1ZAMSs       = Data['SystemParameters']['Mass@ZAMS_1'][()]
    M2ZAMSs       = Data['SystemParameters']['Mass@ZAMS_2'][()]
    
    MZAMStotal    = np.add(M1ZAMS, M2ZAMS)
    
    maskSeedsBecameDCO  = np.in1d(seedsSystems, seedsDCOs)
    totalMassZAMSDCO    = MZAMStotal[maskSeedsBecameDCO]
    
    Data.close()
    return totalMassZAMSDCO

In [ ]:
# calculate function run time
start   = time.time()
MtotNew = calculateTotalMassesNaive(pathData=pathToData)
end     = time.time()
timeDiffOptimized = end-start

# calculate number of Double Compact Objects
nrDCOs = len(Data['DoubleCompactObjects']['SEED'][()])

print('Compare')
print('%s seconds, using Optimizations.' %(timeDiffOptimized)) 
print('%s seconds, using For Loops.'     %(timeDiffNaive)) 
print('Using %s DCO systems'             %(nrDCOs))

*Note:* The time difference will depend on the number of systems under investigation, as well as the number of bypassed For Loops.

In [ ]:
# test that the two arrays are in fact identical
print(np.array_equal(MtotOld, MtotNew))

Note that the above loop can easily be expanded with more conditions.

If you do not want all the DCO initial total masses but only of the double neutron stars, then you just need to apply another mask to the seedsDCOs.

In [ ]:
def calculateTotalMassesDNS(pathToData=None):
    Data  = h5.File(pathToData)
    
    totalMasses = []
    
    #for syntax see section 1 with basic syntax
    seedsDCOs     = Data['DoubleCompactObjects']['SEED'][()]
    type1         = Data['DoubleCompactObjects']['Stellar_Type_1'][()]
    type2         = Data['DoubleCompactObjects']['Stellar_Type_2'][()]
    maskDNS       = (type1 == 13) & (type2 == 13)
    seedsDNS      = seedsDCOs[maskDNS]
    
    #get info from ZAMS
    seedsSystems  = Data['SystemParameters']['SEED'][()]
    M1ZAMSs       = Data['SystemParameters']['Mass@ZAMS_1'][()]
    M2ZAMSs       = Data['SystemParameters']['Mass@ZAMS_2'][()]
    
    MZAMStotal    = np.add(M1ZAMS, M2ZAMS)
    
    
    maskSeedsBecameDNS  = np.in1d(seedsSystems, seedsDNS)
    totalMassZAMSDNS    = MZAMStotal[maskSeedsBecameDNS]
    
    Data.close()
    return totalMassZAMSDNS

In [ ]:
# calculate function run time
start   = time.time()
MtotDNS = calculateTotalMassesDNS(pathToData=pathToData)
end     = time.time()
timeDiffDNS = end-start

# calculate number of DNS systems
nrDNSs = len(MtotDNS)
    
print('%s seconds for all %s DNS systems.' %(timeDiffDNS, nrDNSs)) 

### Example 2

The previous example uses the fact that both SystemParameters and DoubleCompactObjects only print at most one line per system. However, as mentioned above, events such as supernovae or common envelopes might happen multiple times to a given system, and as a result there would be multiple occurences of a given seed in the relevant file. 

To account for this, we will need to modify the previous method. Consider again the 4 seeds of the previous example. Both 2 and 4 formed a DCO and hence both stars in these binaries went SN. Seeds 1 and 3 are low mass stars hence they did not go SN. (Note that we do not specify the companion masses for any of these systems, but for simplicity we assume that the companions to 1 and 3 are also sufficiently low mass to not produce a supernova). The SN file prints one line per SN and therefore seeds 2 and 4 appear twice each.

Imagine you want the primary masses of systems that experienced at any point a core collapse supernova (CCSN). We'll reuse our mock data, with additional information about the types of SN which occured in each star. Here, PPISN refers to Pulsational Pair Instability Supernovae.

In [ ]:
# example mock data from above
SystemSeeds = np.array([1,  2,  3,  4 ])
SystemMass1 = np.array([1, 20,  5, 45 ])
DCOSeeds    = np.array([    2,      4 ])

SNSeeds     = np.array([     2,      2,      4,       4 ])  
SNTypes     = np.array(['CCSN', 'CCSN', 'CCSN', 'PPISN' ])

# get seeds which had a CCSN
maskCCSN  = SNTypes == 'CCSN'
seedsCCSN = SNSeeds[maskCCSN]
print('CCSN seeds =%s' %(seedsCCSN))

#compare which element of 1-d array are in other
#this because in 

seedsCCSN = np.unique(seedsCCSN)
# in this particular case, it is not necessary to reduce seedsCCSN to it's unique entries.
# the numpy.in1d function will work with duplicate seeds, but we include it explicitly here
# as other more complicated scenarios might rely on unique sets of seeds

mask = np.in1d(SystemSeeds, seedsCCSN)
print(SystemMass1[mask])

In [ ]:
# Always remember to close your data file
Data.close()
# ---
# jupyter:
#   jupytext:
#     formats: ipynb,py:light
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.12.0
#   kernelspec:
#     display_name: Python 3
#     language: python
#     name: python3
# ---

# 3. Visualizing the data

Although math is the fundamental basis of physics and astrophysics, we cannot always easily convert numbers and equations into a coherent picture. Plotting is therefore a vital tool in bridging the gap between raw data and a deeper scientific understanding. 

*Disclaimer:*

There are many ways to make the same plot in matplotlib and there are many ways to bin your data. Often, there is no "best" way to display data in a plot, and the message conveyed can be heavily dependent on the context of the data as well as asthetic plotting decisions.

For example, in histograms, as we discuss below, the relatively subjective choice of bin size can significantly affect the interpretation of the results. It is important to be aware of when and how we make these choices and to try to reduce any unintended bias.

---

** Example: inspect the component masses of Double Compact Objects**

In the example below, we use the following conventions:

1 - We deliberately choose to use the matplotlib.pyplot.subplots routine even when creating a single figure (as opposed to using pyplot.plot). This is because many online forums (e.g Stackoverflow) use this syntax. Furthermore, this means you do not have to learn two different types of syntax when creating either a single or multiple panel figure.

2 - We choose to do the binning within the numpy/array environment instead of with inbuilt functions such as plt.hist / axes.hist. The reason is that you have more control over what you do, such as custom normalization (using rates, weights, pdf, etc.). It also forces you to have a deeper understanding of what you are calculating, and allows you to check intermediate steps with print statements.  Once you know how to bin your data this way you can also easily expand these routines for more complicated plots (2D binning).

# Path to be set by user


In [ ]:
pathToData = '/home/cneijssel/Desktop/Test/COMPAS_output.h5'

# Get some data to plot

In [ ]:
Data  = h5.File(pathToData)

print(list(Data.keys()))
# DCOs = double compact objects


DCOs = Data['DoubleCompactObjects']

M1   = DCOs['Mass_1'][()]
M2   = DCOs['Mass_2'][()]
Mtot = np.add(M1, M2)

In [ ]:
Data.close()

# Histogram

In [ ]:
# You can use numpy to create an array with specific min, max and interval values
minMtot = 0
maxMtot = max(Mtot)
nBins   = 50

# Number of bin edges is one more than number of bins
binEdges = np.linspace(minMtot, maxMtot, nBins+1)

# What is the value at the center of the bin?
# add each edge of the side of the bin and divide by 2
xvaluesHist  = (binEdges[:-1] + binEdges[1:])/2.

# What is the width of each bin? (an array in general, if the spacing is non-uniform)
binWidths = np.diff(binEdges)


### Set yvalues to the height of the bins

# Create an array of y-values for each x-value
yvalues = np.zeros(len(xvaluesHist))

# Iterate over the bins to calcuate the number of data points per bin
for iBin in range(nBins):
    mask = (Mtot >= binEdges[iBin]) & (Mtot < binEdges[iBin+1])
    yvalues[iBin] = np.sum(mask)

# You can of course apply any mask you like to get the desired histogram    

## Generally, you can calculate the rate per unit x (dy/dx) using
dYdXHist = np.divide(yvalues, binWidths)

# To convert your distribution to a PDF, normalize in y-values:
PDF = np.divide(yvalues, np.sum(yvalues))

# You can then multiply by, e.g, rates/weights to scale the distribution

# CDF

Sometimes we want to know what fraction of the data lies below a given value. To find this, we calculate a Cumulative Distribution Function, or CDF.

In [ ]:
# Question: How many points have a value less than X? 

# Sort the values of interest
MtotSorted = np.sort(Mtot)   

# These values are your xvalues 
xvaluesCDF = MtotSorted

# The CDF is a non-strictly increasing function from 0 to 1 across the range of x values.
# It should increment by 1/len(xvaluesCDF) at each x in the array, and remain constant otherwise.

# Numpy provides several functions that make this very straightforward
nDataPoints = len(xvaluesCDF)
yvalues = np.cumsum(np.ones(nDataPoints))
CDF = yvalues / nDataPoints

# A two panel plot

In [ ]:
# For two panels side by side:
fig, axes = plt.subplots(1,2, figsize=(18,8))

# axes is an array relating to each panel
# panel1 = axes[0]
# panel2 = axes[1]

largefontsize = 18
smallfontsize = 13




### In the left panel, we want to plot the histogram and CDF overlayed
### with the same x-axis, but different y-axes

# Plot the Histogram first
histAxes = axes[0]
histAxes.plot(xvaluesHist, dYdXHist)

histAxes.set_xlabel(r'Mtot [$M_\odot$]', fontsize=smallfontsize)
histAxes.set_ylabel(r'dN/dMtot [$M_\odot^{-1}$]', fontsize=smallfontsize)

# Overlay the CDF with the same x-axis but different y-axis
cdfAxes =  axes[0].twinx()
cdfAxes.plot(xvaluesCDF, CDF, c='r')

# Dont have to do xlabel since they are the same
cdfAxes.set_ylabel('CDF', fontsize=smallfontsize, labelpad=-40)
cdfAxes.tick_params(axis='y', direction='in', pad=-20) # Adjust the CDF axis for clarity in the plot

axes[0].set_title('Total Mass Histogram and CDF', fontsize=largefontsize)




### In the right panel, we want to display a scatterplot of M1 & M2 

axes[1].scatter(M1, M2)
axes[1].set_xlabel(r'M1 [$M_\odot$]', fontsize=smallfontsize)
axes[1].set_ylabel(r'M2 [$M_\odot$]', fontsize=smallfontsize)

axes[1].set_title('Component Masses', fontsize=largefontsize)




### Clean up and display the plot

# You can force plt to pad enough between plots
# such that the labels fit
plt.tight_layout()

# If you want to save the figure, use:
#plt.savefig(pathToSave)

# To produce the plot, always remember to:
plt.show()